In [1]:
using thesis, PRMaps
using StatsPlots
using Pandas
using Healpix
using StatsPlots
using Plots
using LinearAlgebra
using Dates
using HDF5, JLD

import Stripeline as Sl

In [2]:
wmap = read_pickle("../instruments/wmap_instrument.pkl")
planck = read_pickle("../instruments/planck_instrument.pkl")
quijote = read_pickle("../instruments/quijote_instrument.pkl")
swipe = read_pickle("../instruments/lspe_swipe_instrument.pkl")
strip = read_pickle("../instruments/lspe_strip_instrument.pkl")

columns = ["instrument", "frequency", "depth_p", "depth_i"]

instruments = concat([wmap, planck, quijote, swipe, strip])
instruments = instruments[columns]
instruments

,instrument,frequency,depth_p,depth_i
0,WMAP,22.69,294.697795,293.465609
1,WMAP,32.94,438.561419,296.828735
2,WMAP,40.72,500.985656,415.023172
3,WMAP,40.51,475.110192,393.646897
4,WMAP,60.09,554.721784,520.189018
5,WMAP,60.96,493.519146,462.909657
6,WMAP,92.87,783.584095,750.749159
7,WMAP,93.43,876.013202,839.182282
8,WMAP,92.44,926.757510,887.748258
9,WMAP,93.22,899.656576,861.132559


In [3]:
nside = 64
sky_model = "c1s0d0"
t_start = DateTime(2022, 1, 1, 12, 0, 0)

cam_ang = Sl.CameraAngles()
 
setup = PRMaps.Setup(
    sampling_freq_Hz = 50,
    total_time_s = 24. * 3600. * 10.
    )
nothing

# Run simulation without the error

In [4]:
result = run_fgbuster(instruments, cam_ang, setup, sky_model, nside, t_start)
nothing

In [5]:
FINAL_RESULTS = Dict("ideal"=>result)

Dict{String, Dict{Any, Any}} with 1 entry:
  "ideal" => Dict("nit"=>8, "x"=>[1.54989, 19.6184, -2.99867], "hess_inv"=>[0.0…

# Run simulation with pointing error

In [6]:
labels = ["altitude_offset" "ground_offset" "fork" "zVAX&ω_0" "zVAX&ω_90" "zVAX&ω_180" "zVAX&ω_270"]
error = deg2rad(1.0/60.0)

w2_TA = Sl.TelescopeAngles(wheel2ang_0_rad = error)
w3_TA = Sl.TelescopeAngles(wheel3ang_0_rad = error)
fork_TA = Sl.TelescopeAngles(forkang_rad = error)
wω_TA_0 = Sl.TelescopeAngles(zVAXang_rad = error, ωVAXang_rad = deg2rad(0.0))
wω_TA_90 = Sl.TelescopeAngles(zVAXang_rad = error, ωVAXang_rad = deg2rad(90.0))
wω_TA_180 = Sl.TelescopeAngles(zVAXang_rad = error, ωVAXang_rad = deg2rad(180.0))
wω_TA_270 = Sl.TelescopeAngles(zVAXang_rad = error, ωVAXang_rad = deg2rad(270.0))

tel_angs = [w2_TA, w3_TA, fork_TA, wω_TA_0, wω_TA_90, wω_TA_180, wω_TA_270]

nothing

In [12]:
for (i, tel_ang) in enumerate(tel_angs) 
    FINAL_RESULTS[labels[i]] = run_fgbuster_with_error(instruments, cam_ang, tel_ang, setup, sky_model, nside, t_start)
    println(labels[i]*" Completato!")
end

altitude_offset Completato!


ground_offset Completato!


fork Completato!


zVAX&ω_0 Completato!


zVAX&ω_90 Completato!


zVAX&ω_180 Completato!


zVAX&ω_270 Completato!


In [13]:
# Salvare il dizionario con tutti i risultati